In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from datasets import Dataset, load_from_disk
import pytorch_lightning as pl
import sys
import os
# Add the directory containing lit_sam_model.py to the Python path
sys.path.append(os.path.abspath("../"))
from model.adapterModel import LitSamModel, SAMDataset
from utils.statistics import calculate_correlation
from helperFunctions import *

In [ ]:
import yaml
import os
from pathlib import Path

# 1. Get the path of the script
current_file = Path(__file__).resolve() # src/training/your_script.py

# 2. Go up one level to 'src', then into 'config'
config_path = current_file.parent.parent / "config" / "config_general.yaml"

# 3. Load the YAML
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# 4. Resolve the root of the project (one level above 'src')
# This ensures that "./data" in the YAML is interpreted relative to the Project_Root
PROJECT_ROOT = current_file.parent.parent.parent
os.chdir(PROJECT_ROOT) 

# Extract paths from YAML
DATA_DIR = config['paths']['data']
CHECKPOINT_DIR = config['paths']['checkpoints']
SAM_CHECKPOINT = config['paths']['sam_checkpoint']

In [ ]:
#Load the test dataset

test_dataset = load_from_disk(os.path.join(DATA_DIR, 'datasetTestMetDemSlopeFiltered'))

In [ ]:
print("Dataset keys:", test_dataset.column_names)

In [ ]:
from transformers import SamModel, SamConfig, SamProcessor
import torch

sam_checkpoint = os.path.join(CHECKPOINT_DIR, "sam-float-adapter-metnormalized-epoch=36-val_loss=0.224-val_iou=0.588.ckpt")

# Create an instance of the model architecture with the loaded configuration
model = LitSamModel.load_from_checkpoint_strictless(sam_checkpoint, model_name="vit-b", normalize = True)
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# set the device to cuda if available, otherwise use cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
#model.to(device)

In [ ]:
def calculate_iou(mask1, mask2):

    # Ensure the masks are PyTorch tensors
    if isinstance(mask1, np.ndarray):
        mask1 = torch.tensor(mask1)
    if isinstance(mask2, np.ndarray):
        mask2 = torch.tensor(mask2)
        
    # Ensure the masks are binary
    mask1 = mask1 > 0
    mask2 = mask2 > 0
    
    # Calculate the intersection and union
    intersection = torch.logical_and(mask1, mask2)
    union = torch.logical_or(mask1, mask2)
    
    # Compute the IoU
    iou = torch.sum(intersection).float() / torch.sum(union).float()
    
    return iou, intersection, union

In [ ]:
from torch.utils.data import DataLoader

# Create an instance of the SAMDataset
test_dataset_sam = SAMDataset(dataset=test_dataset, processor=processor, augment=False)

# Create a DataLoader instance for the validation dataset
test_dataloader = DataLoader(test_dataset_sam, batch_size=5, shuffle=False, num_workers=8)

In [ ]:
from torch.utils.data import DataLoader

# Create an instance of the SAMDataset
test_dataset_sam_meteo = SAMDataset(dataset=test_dataset, processor=processor, augment=False, test_meteo=True)

# Create a DataLoader instance for the validation dataset
test_dataloader_meteo = DataLoader(test_dataset_sam_meteo, batch_size=5, shuffle=False, num_workers=8)

In [ ]:
# Create a trainer
trainer = pl.Trainer(accelerator='gpu', devices=1)

# Run the evaluation
trainer.test(model, dataloaders=test_dataloader)

In [ ]:
# Create a trainer
trainer2 = pl.Trainer(accelerator='gpu', devices=1)

# Run the evaluation
trainer2.test(model, dataloaders=test_dataloader_meteo)

In [ ]:
# Access the stored results
test_results = model.test_results
ground_truth_masks = test_results['ground_truth_masks']
predicted_masks = test_results['predicted_masks']
individual_ious = test_results['individual_ious']
bboxes = test_results['bboxes']

In [ ]:
predicted_masks = np.concatenate(predicted_masks, axis=0)
ground_truth_masks = np.concatenate(ground_truth_masks, axis=0)
bounding_boxes = np.concatenate(bboxes, axis=0)

In [ ]:
results_zero_shot = generate_results(test_dataset, ground_truth_masks, predicted_masks, bounding_boxes)

In [ ]:
# Create a DataFrame from the list of dictionaries
df_finetune = pd.DataFrame(results_zero_shot)

# Save the DataFrame to a file (optional)
#df_finetune.to_pickle('dataframe_name.pkl')

In [ ]:
# Filter out rows with None IoU values
df_filtered = df_finetune.dropna(subset=['iou'])


# Create a boolean mask for rows with bbox (0,0,512,512) using apply
mask_bbox = df_filtered['bbox'].apply(
    lambda b: np.all(np.array(b) == np.array((0, 0, 1024, 1024)))
)

In [ ]:
# Filter the DataFrame to exclude rows with bbox (0,0,512,512)

df_filtered = df_filtered[~mask_bbox]

In [ ]:
#
df_filtered = df_filtered[mask_bbox]

In [ ]:
plot_iou_statistics(df_filtered, model_name='NoMeteo')

In [ ]:
plot_iou_statistics(df_filtered, model_name='Meteo')

In [ ]:
plot_mask_area_vs_iou(df_filtered, model_name='Meteo')

In [ ]:
plot_iou_vs_area_ratio(df_filtered, model_name='No Meteo')

In [ ]:
plot_iou_vs_num_avalanches(df_filtered, model_name='No Meteo')

In [ ]:
plot_iou_for_mask_area(df_filtered, model_name='No Meteo')

In [ ]:
print(df_filtered['mask_area'].min())

In [ ]:
df_filtered['mask_area'] = df_filtered['mask'].apply(lambda mask: np.sum(np.array(mask, dtype=np.float32)))

In [ ]:
compute_mask_area_iou_correlation(df_filtered, calculate_correlation, model_name='No Meteo')

In [ ]:
compute_mask_area_iou_correlation(df_filtered, calculate_correlation, model_name='No Meteo', scale='log')

In [ ]:
compute_num_avalanche_iou_correlation(df_filtered, calculate_correlation, model_name='No Meteo')

In [ ]:
compute_area_ratio_iou_correlation(df_filtered, calculate_correlation, model_name='No Meteo')

In [ ]:
compute_area_ratio_iou_correlation(df_filtered, calculate_correlation, model_name='No Meteo', scale='log')

In [ ]:
# Save the DataFrame to a file (optional)
df_filtered.to_pickle('dataframe_bestadaptersmodel.pkl')

In [ ]:
df_finetune = df_finetune = pd.read_pickle('dataframe_bestadaptersmodel.pkl')

In [ ]:
results_zero_shot = compute_error_percentages(results_zero_shot, threshold=0.3)

In [ ]:
plot_false_pos_neg_percentages(results_zero_shot,treshold= 0.3, model_name = 'No Meteo')